In [26]:
import numpy as np
from scipy.io import loadmat
import scipy.constants as cst
import matplotlib.pyplot as plt
from sympy.physics.wigner import wigner_3j, wigner_6j, wigner_9j

In [27]:
time_of_flight = 100 # microseconds
switching_freq = 15/100 # MHz
no_switches = np.floor(time_of_flight*switching_freq)
t_step = time_of_flight*1000/no_switches # ns

physical constants

In [28]:
hbar = cst.hbar
k_B = cst.k
c = cst.c
eps_0 = cst.epsilon_0
Gamma = 1/99 # 1/lifetime of the excited state [GHz]
a0 = cst.physical_constants['Bohr radius'][0] # m
q_e = cst.e
r_expval = 0.32*a0
mu_r_expval = 5.22*r_expval
B_0 = 6.68667*1e9 # Hz
T = 7 # K
r_expval=0.32*a0 # [m]
r_expval_m=5.22*r_expval

In [29]:
def delta_kr(i,j):
    if i == j:
        return 1
    else:
        return 0

In [30]:
def ThreeJSymbol(j1,m1,j2,m2,j3,m3):
    """
    Purely here for compatibility of direct translation of Konrad's matlab code.
    """
    return float(wigner_3j(j1,j2,j3,m1,m2,m3))

def SixJSymbol(j1,j2,j3,j4,j5,j6):
    """
    Purely here for compatibility of direct translation of Konrad's matlab code.
    """
    return float(wigner_6j(j1,j2,j3,j4,j5,j6))

def NineJSymbol(j1,j2,j3,j4,j5,j6,j7,j8,j9):
    """
    Purely here for compatibility of direct translation of Konrad's matlab code.
    """
    return float(wigner_9j(j1,j2,j3,j4,j5,j6,j7,j8,j9))

In [31]:
def generate_states(J_ground, J_excited, parity):
    """
    P=0 - e, P=1 - f (parity type of the excited state); Ground -> X1Sigma+, Excited B3Pi1
    """
    G = []
    E = []
    for jg in J_ground:
        if jg == 0:
            # (X or B,J,W,F1,F,mF,parity type) 
            G=[[0,0,0,1/2,0,0,0],[0,0,0,1/2,1,-1,0],[0,0,0,1/2,1,0,0],[0,0,0,1/2,1,1,0]] 
        else:
            W = 0
            for F1 in [-1/2, 1/2]:
                for F2 in [-1/2, 1/2]:
                    F = jg+F1+F2
                    for mF in np.arange(-F,F+1):
                        G.append([0,jg,W,jg+F1,F,mF,0])
    for je in J_excited:
        if je == 0:
            # (X or B, J,|W|,F1,F,mF,parity) 
            E=[[1,0,1,1/2,0,0,parity], [1,0,1,1/2,1,-1,parity], [1,0,1,1/2,1,0,parity], [1,0,1,1/2,1,1,parity]]  
        else:
            W = 1
            for F1 in [-1/2,1/2]:
                for F2 in [-1/2,1/2]:
                    F = je+F1+F2
                    for mF in np.arange(-F,F+1):
                        E.append([1,je,W,je+F1,F,mF,parity])
    return G,E

In [32]:
def dipoleHelper(S, Mixing_M, ind):
    st = []
    if S[0] > 0:
        if S[1] == 1:
            for i in range(0,8):
                if Mixing_M[ind, i] != 0:
                    if i == 0:
                        st.append([1, S[2], 1/2, 0, S[5], i])
                    elif i == 1:
                        st.append([1, S[2], 1/2, 1, S[5], i])
                    elif i == 2:
                        st.append([1, S[2], 3/2, 1, S[5], i])
                    elif i == 3:
                        st.append([1, S[2], 3/2, 2, S[5], i])
                    elif i == 4:
                        st.append([2, S[2], 3/2, 1, S[5], i])
                    elif i == 5:
                        st.append([2, S[2], 3/2, 2, S[5], i])
                    elif i == 6:
                        st.append([2, S[2], 5/2, 2, S[5], i])
                    elif i == 8:
                        st.append([3, S[2], 5/2, 2, S[5], i])
        elif S[1] == 2:
            for i in range(0,11):
                if Mixing_M[ind, i] != 0:
                    if i == 0:
                        st.append([1, S[2], 1/2, 1, S[5], i])
                    elif i == 1:
                        st.append([1, S[2], 3/2, 1, S[5], i])
                    elif i == 2:
                        st.append([1, S[2], 3/2, 2, S[5], i])
                    elif i == 3:
                        st.append([2, S[2], 3/2, 1, S[5], i])
                    elif i == 4:
                        st.append([2, S[2], 3/2, 2, S[5], i])
                    elif i == 5:
                        st.append([2, S[2], 5/2, 2, S[5], i])
                    elif i == 6:
                        st.append([2, S[2], 5/2, 3, S[5], i])
                    elif i == 7:
                        st.append([3, S[2], 5/2, 2, S[5], i])
                    elif i == 8:
                        st.append([3, S[2], 5/2, 3, S[5], i])
                    elif i == 9:
                        st.append([3, S[2], 7/2, 3, S[5], i])
                    elif i == 10:
                        st.append([4, S[2], 7/2, 3, S[5], i])
    else:
        st.append([S[1], S[2], S[3], S[4], S[5]])
    return st

In [33]:
def dipoleTransitionMatrixElement(E,G,p,mixing):
    """
    Wtf is p? Konrads code has absolutely no comments anywhere
    """
    if not mixing:
        Je, We, F1e, Fe, Me, Pe = E[1:]
        Jg, Wg, F1g, Fg, Mg, Pg = G[1:]
        
        power = Fe-Me+Fg+F1g+F1e+2*Je-We+3
        parg = (-1)**(Jg+Pg)
        pare = (-1)**(Je+Pe)
        delta = 1-delta_kr(parg,pare)
            
        s = 0
        for q in [-1,0,1]:
            if (Wg != 0) & (We != 0):
                s += ThreeJSymbol(Je,-We,1,q,Jg,Wg)+(-1)**Pe*ThreeJSymbol(Je,We,1,q,Jg,Wg)+(-1)**Pg*ThreeJSymbol(Je,-We,1,q,Jg,-Wg)+(-1)**(Pg+Pe)*ThreeJSymbol(Je,We,1,q,Jg,-Wg)
            elif (Wg == 0) & (We != 0):
                s += np.sqrt(2)*(ThreeJSymbol(Je,-We,1,q,Jg,Wg)+(-1)**Pe*ThreeJSymbol(Je,We,1,q,Jg,Wg))
            elif (Wg != 0) & (We == 0):
                s += np.sqrt(2)*(ThreeJSymbol(Je,-We,1,q,Jg,Wg)+(-1)**Pg*ThreeJSymbol(Je,-We,1,q,Jg,-Wg));
            else:
                s += 2*ThreeJSymbol(Je,-We,1,q,Jg,Wg)
        x = ((-1)**power)*delta*s*np.sqrt((2*Fg+1)*(2*Jg+1)*(2*Fe+1)*(2*Je+1)*(2*F1g+1)*(2*F1e+1)) \
                         *ThreeJSymbol(Fe,-Me,1,p,Fg,Mg) \
                         *SixJSymbol(Jg,F1g,1/2,F1e,Je,1) \
                         *SixJSymbol(F1g,Fg,1/2,Fe,F1e,1)*1/2
    if mixing:
        x = 0
        
        if ((E[0] > 0) & (E[1] == 1)) | ((G[0] > 0) & (G[1] == 1)):
            Mixing_M = np.zeros([4,8])
            Mixing_M[0,:] = [1,0,0,0,0,0,0,0] # E. Norrgard et al.
            Mixing_M[1,:] = [0,0.9996,0.0203,0,0.018,0,0,0]
            Mixing_M[2,:] = [0,0.0267,-0.8519,0,-0.5231,0,0,0]
            Mixing_M[3,:] = [0,0,0,0.8483,0,0.5293,0.0138,0.0064]
        elif ((E[0] > 0) & (E[1] == 2)) | ((G[0] > 0) & (G[1] == 2)):
            Mixing_M = np.zeros([4,11]);  # |J,F1,F>: |1,1/2,1>,|1,3/2,1>,|1,3/2,2>,|2,3/2,1>,|2,3/2,2>,|2,5/2,2>,|2,5/2,3>,|3,5/2,2>,|3,5/2,3>,|3,7/2,3>,|4,7/2,3>
            Mixing_M[0,:] = [-0.0048,-0.5235,0,0.8521,0,0,0,0,0,0,0]
            Mixing_M[1,:] = [0,0,0.5294,0,-0.8483,-0.0011,0,-0.0103,0,0,0]
            Mixing_M[2,:] = [0,0,0.0103,0,0.012,-0.9353,0,-0.3534,0,0,0]
            Mixing_M[3,:] = [0,0,0,0,0,0,0.9342,0,0.3568,0.01,0.0032]
            
        e_ind = 0
        g_ind = 0
        if E[1] == 1:
            if E[3] == 1/2:
                e_ind = 0 if E[4] == 0 else 1
            else:
                e_ind = 2 if E[4] == 1 else 3
        elif E[1] == 2:
            if E[3] == 3/2:
                e_ind = 0 if E[4] == 1 else 1
            else:
                e_ind = 2 if E[4] == 2 else 3

        if G[1] == 1:
            if G[3] == 1/2:
                g_ind = 0 if G[4] == 0 else 1
            else:
                g_ind = 2 if G[4] == 1 else 3
        elif G[1] == 2:
            if G[3] == 3/2:
                g_ind = 0 if G[4] == 1 else 1
            else:
                g_ind = 2 if G[4] == 2 else 3
        
        
        E_st = dipoleHelper(E, Mixing_M, e_ind)
        G_st = dipoleHelper(G, Mixing_M, g_ind)
        
        for i,e_st in enumerate(E_st):
            for j,g_st in enumerate(G_st):
                Je, We, F1e, Fe, Me = e_st[:5]
                Jg, Wg, F1g, Fg, Mg = g_st[:5]
                Pe_mixed = E[6]
                Pg_mixed = E[6]
                                
                Je_mixed = E[1]
                Jg_mixed = G[1]
                
                Pe = Je-Je_mixed+Pe_mixed % 2
                Pg = Jg-Jg_mixed+Pg_mixed % 2

                power = Fe-Me+Fg+F1g+F1e+2*Je-We+3
                parg = (-1)**(Jg_mixed+Pg_mixed)
                pare = (-1)**(Je_mixed+Pe_mixed)
                delta = 1-delta_kr(parg,pare)
                
                if delta == 0:
                    print('delta = 0')
                    continue
                
                s = 0
                for q in [-1,0,1]:
                    if (Wg != 0) & (We != 0):
                        print('1')
                        s += ThreeJSymbol(Je,-We,1,q,Jg,Wg) \
                           + (-1)**Pe*ThreeJSymbol(Je,We,1,q,Jg,Wg) \
                           + (-1)**Pg*ThreeJSymbol(Je,-We,1,q,Jg,-Wg) \
                           + (-1)**(Pg+Pe)*ThreeJSymbol(Je,We,1,q,Jg,-Wg)
                    elif (Wg == 0) & (We != 0):
                        print('2')
                        s += np.sqrt(2)*(ThreeJSymbol(Je,-We,1,q,Jg,Wg) \
                                         + (-1)**Pe*ThreeJSymbol(Je,We,1,q,Jg,Wg))
                    elif (Wg != 0) & (We == 0):
                        print('3')
                        s += np.sqrt(2)*(ThreeJSymbol(Je,-We,1,q,Jg,Wg) \
                                         + (-1)**Pg*ThreeJSymbol(Je,-We,1,q,Jg,-Wg))
                    else:
                        print('4')
                        s += 2*ThreeJSymbol(Je,-We,1,q,Jg,Wg)
                partial_x = ((-1)**power)*delta*s*np.sqrt((2*Fg+1)*(2*Jg+1)*(2*Fe+1)*(2*Je+1)*(2*F1g+1)*(2*F1e+1)) \
                                         *ThreeJSymbol(Fe,-Me,1,p,Fg,Mg) \
                                         *SixJSymbol(Jg,F1g,1/2,F1e,Je,1) \
                                         *SixJSymbol(F1g,Fg,1/2,Fe,F1e,1)*1/2
                print(partial_x)
                
                if (E[0] > 0) & (G[0] == 0):
                    x += partial_x*Mixing_M[e_ind, e_st[5]]
                elif ((E[0] == 0) & (G[0] > 0)):
                    x += partial_x*Mixing_M[g_ind, g_st[5]]
                elif ((E[0] > 0) & (G[0] > 0)):
                    x += partial_x*Mixing_M[g_ind, g_st[5]]*Mixing_M[e_ind, e_st[5]]
                else:
                    x = partial_x
    return x

In [34]:
def boltzmann_distribution(rot_constant, temperature, J_list):
    init = np.zeros(len(J_list))
    Z = 0
    g = 1
    for J in J_list:
        g = 4*(2*J+1)
        Z += g*np.exp(-rot_constant*J*(J+1)*2*np.pi*hbar/(temperature*k_B))
    for idx, J in enumerate(J_list):
        g = 4*(2*J+1)
        init[idx] = g*np.exp(-rot_constant*J*(J+1)*2*np.pi*hbar/(temperature*k_B))/Z
    return init

### states

In [35]:
J_g = [0,1,3]
J_e = [1]

ground_states, excited_states = generate_states(J_g, J_e, 1)

In [36]:
rabi_matrix = loadmat('TransitionMatrix_EtoE_mixing_full.mat')['rabi_matrix']
rabi_matrix.shape

(112, 112, 3)

In [37]:
# elemination of states that don't participate in the process
# not sure why these particular states were chosen to ignore

# -1 because matlab indexing starts at 1
for i in range(104-1,101-1,-1):
    rabi_matrix[i,:,:] = 0
    rabi_matrix[:,i,:] = 0
for i in range(100-1,37,-1):
    rabi_matrix[i,:,:] = 0
    rabi_matrix[:,i,:] = 0
    
n = 36+3+5

In [38]:
ini = boltzmann_distribution(B_0,T, range(0,20+1))

In [39]:
ic = np.zeros([n,n])
idx = 0
for j in J_g:
    for i in np.arange(0,4*(2*j+1)):
        k = idx+i
        ic[k,k] = ini[j+1]/(4*(2*j+1))
    idx += 4*(2*j+1)
ic /= np.sum(ini[:max(J_g)])

In [40]:
ini

array([4.51479157e-02, 1.23577435e-01, 1.71454290e-01, 1.82312749e-01,
       1.62436032e-01, 1.25525981e-01, 8.55786804e-02, 5.19726668e-02,
       2.82861857e-02, 1.38516340e-02, 6.12024571e-03, 2.44489252e-03,
       8.84371078e-04, 2.90002065e-04, 8.62895774e-05, 2.33144552e-05,
       5.72349052e-06, 1.27725831e-06, 2.59212713e-07, 4.78567584e-08,
       8.04019977e-09])

In [41]:
np.diag(ic)

array([0.09081778, 0.09081778, 0.09081778, 0.09081778, 0.04200092,
       0.04200092, 0.04200092, 0.04200092, 0.04200092, 0.04200092,
       0.04200092, 0.04200092, 0.04200092, 0.04200092, 0.04200092,
       0.04200092, 0.0170536 , 0.0170536 , 0.0170536 , 0.0170536 ,
       0.0170536 , 0.0170536 , 0.0170536 , 0.0170536 , 0.0170536 ,
       0.0170536 , 0.0170536 , 0.0170536 , 0.0170536 , 0.0170536 ,
       0.0170536 , 0.0170536 , 0.0170536 , 0.0170536 , 0.0170536 ,
       0.0170536 , 0.0170536 , 0.0170536 , 0.0170536 , 0.0170536 ,
       0.0170536 , 0.0170536 , 0.0170536 , 0.0170536 ])

In [42]:
from sympy import Symbol

In [43]:
""" Energies """
w1 = Symbol(u'ω_0', real = True)
w2 = Symbol(u'ω_1', real = True)
we = Symbol(u'ω_{e}', real = True)
we2 = Symbol(u'ω_{e2}', real = True)

""" Light Frequencies """
wm0 = Symbol(u'ω_{m0}', real = True)
wm1 = Symbol(u'ω_{m1}', real = True)
wL= Symbol(u'ω_{L}', real = True)
wL2 = Symbol(u'ω_{L2}', real = True)

""" Rabi Frequencies """
Wm0 = Symbol(u'Ω_{m0}', real = True)
Wm1 = Symbol(u'Ω_{m1}', real = True)
WL = Symbol(u'Ω_{L}', real = True)
WL2 = Symbol(u'Ω_{L2}', real = True)

""" Detunings """
dm0 = Symbol(u'Δ_{m0}', real = True)
dm1 = Symbol(u'Δ_{m1}', real = True)
dL = Symbol(u'Δ_{L}', real = True)
dL2 = Symbol(u'Δ_{L2}', real = True)

""" Microwave angles and polarization """
am0 = Symbol(u'α_0', real = True)
pm0 = Symbol(u'p_0', real = True)
am1 = Symbol(u'α_1', real = True)
pm1 = Symbol(u'p_0', real = True)

""" Laser Polarization """
pL = Symbol(u'p_L', real = True)

In [44]:
# Values 
Del_10=22.24*1e-6*2*np.pi
Del_1=0.17595*1e-3*2*np.pi
Del_11=14.54*1e-6*2*np.pi
Del_21=44.52*1e-6*2*np.pi
Del_2=0.27881*1e-3*2*np.pi
Del_22=33.22*1e-6*2*np.pi
Del_32=63.95*1e-6*2*np.pi
Del_3=0.38458*1e-3*2*np.pi
Del_33=54.38*1e-6*2*np.pi
Del_0=13.3*1e-6*2*np.pi
det_L=(Del_2/2+Del_21)
det_m0=0
det_m1=0
det_m3=0
polangle_m0=np.pi/4
dirangle_m0=np.pi/4
polangle_m1=np.pi/4
dirangle_m1=np.pi/4
polangle_m3=np.pi/4
dirangle_m3=np.pi/2

In [45]:
# Light Properties
P_L=0.05 # [W]
diam_L=0.003 # [m]
P_L2=0.00 # [W]
diam_L2=0.003 # [m]
P_m0=0.000 # [W]
diam_m0=0.05 #[m]
P_m1=0.5 # [W]
diam_m1=0.05 # [m]

Rabi_m0=1e-9*q_e*r_expval_m*np.sqrt(8*P_m0/(np.pi*c*eps_0*diam_m0**2))/hbar # [GHz]
Rabi_m1=1e-9*q_e*r_expval_m*np.sqrt(8*P_m1/(np.pi*c*eps_0*diam_m1**2))/hbar # [GHz]
Rabi_L=1e-9*q_e*r_expval*np.sqrt(8*P_L/(np.pi*c*eps_0*diam_L**2))/hbar
Rabi_L2=1e-9*q_e*r_expval*np.sqrt(8*P_L2/(np.pi*c*eps_0*diam_L2**2))/hbar

In [19]:
BR = loadmat('BranchingRatios_EtoE_mixing_full.mat')['BR']

In [20]:
# elemination of states that don't participate in the process
# not sure why these particular states were chosen to ignore

# -1 because matlab indexing starts at 1
for i in range(104-1,101-1,-1):
    BR[i,:] = 0
    BR[:,i] = 0
for i in range(100-1,37,-1):
    BR[i,:] = 0
    BR[:,i] = 0
    
for i in range(0,n-8):
    BR[i,:] = 0

In [21]:
from optical_bloch import Hamiltonian, Dissipator, BlochEquations

In [22]:
L = Dissipator(n)

In [23]:
G = Symbol('G', real = True)
DR = [0]*(n-8)
DR += [G,G,G,G,G,G,G,G]

In [33]:
L.fromBranching(BR,DR)

In [ ]:
syms t pol_mod pol_mod_m real;

# Laser 1
WLp=W_L/sqrt(2)*sin(pi/2*(1+sign(sin(2*pi*switching_freq*t)))/2)*pol_mod;
WLm=W_L/sqrt(2)*sin(pi/2*(1+sign(sin(2*pi*switching_freq*t)))/2)*pol_mod;
WLz=W_L*cos(pi/2*(1+sign(sin(2*pi*switching_freq*t)))/2)*pol_mod+(1-pol_mod)*W_L;


WL_pol=[WLm,WLz,-WLp];

# Laser 2
WLp2=0;
WLm2=0;
WLz2=W_L2;

WL_pol2=[WLm2,WLz2,-WLp2];

# Microwaves
Wm1p=W_m1/sqrt(2)*(sin(p_m1)+1i*cos(a_m1)*cos(p_m1));
Wm1m=W_m1/sqrt(2)*(sin(p_m1)-1i*cos(a_m1)*cos(p_m1));
Wm1z=-W_m1*cos(p_m1)*sin(a_m1);

Wm1_pol=[Wm1m,Wm1z,-Wm1p];

Wm1_pol=subs(Wm1_pol,p_m1,pi/2*(1-pol_mod_m)+pol_mod_m*pi/2*(1+sign(sin(2*pi*switching_freq*t+pi/2)))/2);